<a href="https://colab.research.google.com/github/EhsaasN/LLM-learning/blob/main/QLoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%pip install transformers datasets peft accelerate bitsandbytes

  Using cached datasets-3.5.0-py3-none-any.whl.metadata (19 kB)
  Using cached bitsandbytes-0.45.5-py3-none-manylinux_2_24_x86_64.whl.metadata (5.0 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manyli

In [4]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
import torch

In [5]:
from transformers import BitsAndBytesConfig

In [6]:

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

BEFORE TRAINING

In [7]:
prompt = "### Financial Statement:\nThe company's earnings surpassed expectations this quarter.\n### Sentiment:\n"

In [8]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

In [9]:
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=40,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.9,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id
    )

print(tokenizer.decode(output[0], skip_special_tokens=True))

### Financial Statement:
The company's earnings surpassed expectations this quarter.
### Sentiment:
Highly Positive sentiment from our clients and partners. ### Social Media:
Over 200 mentions in the last week. ### Web Traffic:
Our website traffic has increased by


TRAINING

In [10]:
model = prepare_model_for_kbit_training(model)

In [11]:
# ✅ QLoRA Config
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

In [16]:
model = get_peft_model(model, lora_config)

In [12]:
# ✅ Create a random financial dataset
data = {
    "text": [
        "The stock market closed higher today due to strong earnings reports.",
        "The Federal Reserve raised interest rates by 0.25 basis points.",
        "Apple's quarterly revenue exceeded expectations driven by iPhone sales.",
        "Inflation has slowed down but remains above the central bank's target.",
        "The crypto market experienced a sharp decline amid regulatory concerns."
    ]
}

dataset = Dataset.from_dict(data)

In [13]:
def tokenize_function(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=256
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = dataset.map(tokenize_function)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [17]:
training_args = TrainingArguments(
    output_dir="./tinyllama-financial-qlora",
    per_device_train_batch_size=1,
    num_train_epochs=10,
    logging_dir="./logs",
    logging_steps=5,
    save_strategy="epoch",
    learning_rate=2e-4,
    fp16=True,
    report_to="none"
)

In [18]:
# ✅ Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

# ✅ Start fine-tuning
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
5,7.495600
10,7.202200
15,4.764100
20,2.156300
25,0.887400
30,0.380800
35,0.217600
40,0.176400
45,0.164200
50,0.158100


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

TrainOutput(global_step=50, training_loss=2.360273687839508, metrics={'train_runtime': 24.2025, 'train_samples_per_second': 2.066, 'train_steps_per_second': 2.066, 'total_flos': 79537058611200.0, 'train_loss': 2.360273687839508, 'epoch': 10.0})

In [19]:
model.save_pretrained("./tinyllama-financial-qlora")
tokenizer.save_pretrained("./tinyllama-financial-qlora")

('./tinyllama-financial-qlora/tokenizer_config.json',
 './tinyllama-financial-qlora/special_tokens_map.json',
 './tinyllama-financial-qlora/tokenizer.model',
 './tinyllama-financial-qlora/added_tokens.json',
 './tinyllama-financial-qlora/tokenizer.json')

TESTING

In [20]:
base_model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
fine_tuned_path = "./tinyllama-financial-qlora"

In [21]:
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

In [22]:
from peft import PeftModel

In [23]:
model = PeftModel.from_pretrained(base_model, fine_tuned_path)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_path)
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 2048)
        (layers): ModuleList(
          (0-21): 22 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear

In [24]:
prompt = "### Financial Statement:\nThe company's earnings surpassed expectations this quarter.\n### Sentiment:\n"

In [25]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

In [26]:
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=20,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.9,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id
    )

print(tokenizer.decode(output[0], skip_special_tokens=True))

### Financial Statement:
The company's earnings surpassed expectations this quarter.
### Sentiment:
Analysts are optimistic about the company's future growth and profitability.


In [27]:
prompt="### Financial Statement:\nThe company's profits has been decreasing for the past few months. \n### Sentiment:\n"

In [28]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

In [29]:
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=40,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.9,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id
    )

print(tokenizer.decode(output[0], skip_special_tokens=True))

### Financial Statement:
The company's profits has been decreasing for the past few months. 
### Sentiment:
The sentiment of the market is negative, and there is a low demand for stocks. The declining profitability has led to a decrease in the sentiment towards the company.
